In [1]:
import os
import pathlib
ospath=os.environ['PATH']
os.environ['PATH']=str(pathlib.PurePath(r'C:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\include'))+os.environ['PATH']
os.environ['PATH']=str(pathlib.PurePath(r'C:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\bin'))+os.environ['PATH']
os.environ['PATH']=str(pathlib.PurePath(r'C:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\lib\x64'))+os.environ['PATH']
os.environ['PATH']=str(pathlib.PurePath(r'C:\Users\q\Anaconda3\pkgs\cudatoolkit-10.0.130-0\DLLs'))+os.environ['PATH']
os.environ['PATH']=str(pathlib.PurePath(r'C:\Users\q\Anaconda3\pkgs\cudatoolkit-10.0.130-0\Library\bin'))    +os.environ['PATH']
print(os.environ['PATH'])

C:\Users\q\Anaconda3\pkgs\cudatoolkit-10.0.130-0\Library\binC:\Users\q\Anaconda3\pkgs\cudatoolkit-10.0.130-0\DLLsC:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\lib\x64C:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\binC:\Users\q\Anaconda3\pkgs\cudnn-7.6.4-cuda10.0_0\Library\includeC:\Users\q\Anaconda3;C:\Users\q\Anaconda3\Library\mingw-w64\bin;C:\Users\q\Anaconda3\Library\usr\bin;C:\Users\q\Anaconda3\Library\bin;C:\Users\q\Anaconda3\Scripts;C:\Users\q\Anaconda3\bin;C:\Users\q\Anaconda3\condabin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\Git\cmd;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Users\q\AppData\Local\Microsoft\WindowsApps;C:\Users\q\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\q\AppData\Local\Programs\Python\Python37;C:\Users\q\AppData\Local\Programs\Python\Python37\Scripts;C:\MinGW\bin

In [2]:
"""
2019-11-30

오리지널 주소 : https://www.tensorflow.org/tutorials/images/cnn
위부분에 path지정하는곳 을 먼저 실행함.
https://stackoverflow.com/a/59007505 에 나온 소스를 추가함.

windows 10, anaconda환경에서 작업.
cudatoolkit : 10.0.130
cudnn : 7.6.4
tensorflow : 2.0.0
tensorflow-gpu : 2.0.0
gpu : GTX 960 2GB

"""
#https://www.tensorflow.org/tutorials/images/cnn
from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0


#https://stackoverflow.com/a/59007505 : Failed to get convolution algorithm. 오류
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

#tf.config.gpu.set_per_process_memory_growth(True)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 960, pci bus id: 0000:01:00.0, compute capability: 5.2



In [3]:
#합성곱층 만들기
"""
cnn 입력 : tensor : 이미지 눂이, 너비, 컬러
"""
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary() #모델으 구조

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
#dense 층 추가.
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [5]:


#모델 컴파일과 훈련하기
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 13s 220us/sample - loss: 0.1468 - accuracy: 0.9544


In [6]:
#모델 평가
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

10000/1 - 1s - loss: 0.0241 - accuracy: 0.9844
